In [14]:
from sklearn.manifold import TSNE
import plotly.express as px
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
import pl_bolts
import os

from torch.utils.data import Dataset, DataLoader, random_split
from lightly.data import LightlyDataset
from glob import glob
from PIL import Image
from sklearn.manifold import TSNE

from pl_bolts.models.self_supervised.simclr import SimCLRTrainDataTransform, SimCLREvalDataTransform
from torchvision.transforms import Compose, ToTensor, Normalize
from lightly.transforms.mae_transform import MAETransform

In [7]:
transforms_dict = {
    'simclr': {
        'train': SimCLRTrainDataTransform(),
        'val': SimCLREvalDataTransform(),
        'test': SimCLREvalDataTransform()
    },
    'mae': {
        'train': MAETransform(),
        'val': Compose([ToTensor(), Normalize(0.5, 0.5)]),
        'test': Compose([ToTensor(), Normalize(0.5, 0.5)])
    },
    'byol': {
        'train': SimCLRTrainDataTransform(),
        'val': SimCLREvalDataTransform(),
        'test': SimCLREvalDataTransform()
    },
    'downstream_linear': {
        'train': SimCLREvalDataTransform(),
        'val': SimCLREvalDataTransform(),
        'test': SimCLREvalDataTransform()
    }
}


def transform_factory(model_name, mode):
    try:
        return transforms_dict.get(model_name).get(mode)
    except KeyError:
        raise NotImplementedError(f'{model_name} {mode} transform not implemented')


In [8]:
class ICDARDataset(Dataset):

    def __init__(self, csv_filepath, root_dir, transforms=None, convert_rgb=True, mask_generator=None):
        self.root_dir = root_dir
        self.transforms = transforms
        self.data = pd.read_csv(csv_filepath, sep=';')
        self.convert_rgb = convert_rgb
        self.mask_generator = mask_generator

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(self.root_dir, self.data.loc[idx, 'FILENAME'])
        
        try:
            image = Image.open(img_path)
        except Exception as ex:
            return None

        if self.convert_rgb:
            image = image.convert('RGB')
        
        if self.transforms is not None:
            image = self.transforms(image)
        
        return image, self.data.loc[idx, 'SCRIPT_TYPE']


In [9]:
def data_factory(dataset_name, root_dir, label_filepath, transforms, mode, batch_size, collate_fn=None, num_cpus=None):

    if dataset_name.lower() == 'icdar':
        dataset = ICDARDataset(label_filepath, root_dir, transforms=transforms, convert_rgb=True)
    elif dataset_name.lower() == 'icdar_lightly':
        dataset = LightlyDataset(input_dir=root_dir, transform=transforms, filenames=glob(root_dir + '/*.tif'))
    else:
        raise NotImplementedError(f'Dataset {dataset_name} is not implemented')

    total_count = len(dataset)
    train_count = int(0.7 * total_count)
    val_count = int(0.1 * total_count)
    test_count = total_count - train_count - val_count

    train_dataset, val_dataset, test_dataset = random_split(
        dataset,
        (train_count, val_count, test_count)
    )

    if mode in 'train':
        return {
            'train': DataLoader(
                train_dataset,
                batch_size=batch_size,
                shuffle=True,
                drop_last=True,
                pin_memory=True,
                num_workers=num_cpus or os.cpu_count(),
                collate_fn=collate_fn if collate_fn else None
            ),
            'val': DataLoader(
                val_dataset,
                batch_size=batch_size,
                shuffle=False,
                drop_last=False,
                pin_memory=True,
                num_workers=num_cpus or os.cpu_count(),
                collate_fn=collate_fn if collate_fn else None
            )
        }
    elif mode == 'test':
        return {
            'test': DataLoader(
                test_dataset,
                batch_size=batch_size,
                shuffle=False,
                drop_last=False,
                pin_memory=True,
                num_workers=os.cpu_count(),
                collate_fn=collate_fn if collate_fn else None
            )
        }
    else:
        raise KeyError(f'Unknown mode: {mode}')


In [17]:
def plot_features(model, data_loader, num_feats, batch_size, num_samples):
    num_samples = len(data_loader) if not num_samples else num_samples
    feats = np.array([]).reshape((0, num_feats))
    labels = np.array([])
    model.eval()
    model.cuda()

    processed_samples = 0
    with torch.no_grad():
        for (x1, x2, _), label in data_loader:
            if processed_samples >= num_samples:
                break
            x1 = x1.squeeze().cuda()
            out = model(x1)
            out = out.cpu().data.numpy()
            print(out.shape)
            feats = np.append(feats, out, axis=0)
            labels = np.append(labels, label, axis=0)
            processed_samples += batch_size

    tsne = TSNE(n_components=3, perplexity=30, init='pca')
    x_feats = tsne.fit_transform(feats)

    dim_red_df = pd.DataFrame(x_feats)
    dim_red_df['labels'] = pd.Categorical(labels)
    fig = px.scatter_3d(dim_red_df, x=0, y=1, z=2, color='labels')
    fig.show()


In [18]:
checkpoint = '/home/woody/iwfa/iwfa028h/dev/faps/data/trained_models/SimCLR/lightning_logs/version_584360/checkpoints/epoch=474-step=4275.ckpt'

MODEL_NAME = 'simclr'
MODE = 'test'
BATCH_SIZE = 64

ssm = pl_bolts.models.self_supervised.SimCLR.load_from_checkpoint(checkpoint)

transforms = transform_factory(MODEL_NAME, MODE)

data = data_factory(
    'icdar',
    '/home/woody/iwfa/iwfa028h/dev/faps/data/ICDAR2017_CLaMM_Training',
    '/home/woody/iwfa/iwfa028h/dev/faps/data/ICDAR2017_CLaMM_Training/@ICDAR2017_CLaMM_Training.csv',
    transforms,
    'train',
    BATCH_SIZE,
    collate_fn=None,
    num_cpus=None
)

plot_features(
    ssm,
    data.get('train'),
    2048,
    BATCH_SIZE,
    500
)

/home/hpc/iwfa/iwfa028h/.conda/envs/lme/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:224: UnderReviewWarning:

The feature SimCLR is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html

/home/hpc/iwfa/iwfa028h/.conda/envs/lme/lib/python3.10/site-packages/pl_bolts/models/self_supervised/simclr/simclr_module.py:138: UnderReviewWarning:

The feature resnet50 is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html

/home/hpc/iwfa/iwfa028h/.conda/envs/lme/lib/python3.10/site-packages/pl_bolts/models/self_supervised/resnets.py:3

(64, 2048)
(64, 2048)
(64, 2048)
(64, 2048)
(64, 2048)
(64, 2048)
(64, 2048)
(64, 2048)
